In [ ]:
import pandas as pd
import plotly.express as px

def plot_ts(df, title, x_title, y_title, width, height):
    fig = px.line(df, title=title, width=width, height=height)
    fig.update_layout(
        xaxis_title=x_title,
        yaxis_title=y_title,
    )
    fig.show()

### Read Data

In [ ]:
df_state = pd.read_csv("/kaggle/input/covid-vaccination-india-district-wise-data/vaccine_doses_statewise.csv")
df_state_cowin = pd.read_csv("/kaggle/input/covid-vaccination-india-district-wise-data/cowin_vaccine_data_statewise.csv")
df_distric_cowin = pd.read_csv("/kaggle/input/covid-vaccination-india-district-wise-data/cowin_vaccine_data_districtwise.csv", low_memory=False)
df_pop = pd.read_csv("/kaggle/input/covid-vaccination-india-district-wise-data/india_state_wise_projected_population_2021.csv")

df_pop =  df_pop.rename(columns={"State Name": "State", "Total Population(Projected 20201)": "Population"})

### Take a look of the data

#### Following data represents number of vaccine doses administered state wise over time, This data has been collected from MOHFW daily bulletin

In [ ]:
df_state.head()

### State wise vaccination data collected from CoWin Website

In [ ]:
df_state_cowin.head()

### District wise vaccination data collected from CoWin Website

In [ ]:
df_distric_cowin.head()

### Process Data for visualization

In [ ]:
df_state = df_state[:-2]
df_state_trans = df_state.T
df_state_trans = df_state_trans.reset_index(drop=False)
column_names = list(df_state_trans.iloc[0].values)
column_names.remove("State")
column_names = ["Date"] + column_names
df_state_trans.columns = column_names
df_state_trans = df_state_trans.drop(0)
df_state_trans.Date = pd.to_datetime(df_state_trans.Date, format="%d/%m/%Y")
df_state_trans = df_state_trans.set_index("Date")

for name in df_state_trans.select_dtypes("object"):
    df_state_trans[name] = df_state_trans[name].astype('int')

# Plot absolute number of vaccines administered in different states over time

In [ ]:
plot_ts(df_state_trans,title="Absolute Number of Vaccines Administered", x_title="Number of doses", y_title="Date", width=1000, height=600)

# Plot percentage population vaccinated

Considering, every individual needs 2 doses of vaccine (This is true for Covishield, Covaxin or Sputnik), India needs around 2.8 Billion doses of Vaccines (including children between age 0 to 18) to vaccinate it's entire population. If the shots acquired so far has been used to doubly vaccinate people what percentage of population would have been vaccinated?

This may not be an ideal way to track the progress made. But, of course gives some rough idea.

In [ ]:
df_state_trans_percent = pd.DataFrame()
for state_name in df_state_trans.columns:
    pop = df_pop[df_pop["State"] == state_name]["Population"].values[0]
    if pop is not None:
        df_state_trans_percent[state_name] = (df_state_trans[state_name]/ (2 * pop)) * 100
        
plot_ts(df_state_trans_percent,title="Percentage of Population That Should Have Been Doubly Vaccinated With Acquired Doses", x_title="Percentage of population", y_title="Date", width=1000, height=600)

### Observation
- Smaller states are performing better

# Plot Percentage of Population with single dose

# Plot Percentage of Population with two doses